In [148]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import f1_score

In [2]:
bach_cont = pd.read_csv("bachelor-contestants.csv")
ette_cont = pd.read_csv("bachelorette-contestants.csv")

In [3]:
elim_data = pd.read_csv('bachelorette_538.csv')
# remove first line, which is just the header info repeated
elim_data = elim_data.drop([0])

# need to change from wide form data to long form data
# want it to be:
# SHOW, SEASON, CONTESTANT, 'TYPE', 'VALUE'
# with 'TYPE' as the current column names
elim1 = pd.melt(elim_data, id_vars=['SHOW', 'SEASON', 'CONTESTANT'])

# rename variable and value
elim1.columns = ['SHOW', 'SEASON', 'CONTESTANT', 'TYPE', 'VALUE']

# split TYPE variable
elim1[['ELIM_DATE', 'NUMBER']] = elim1['TYPE'].str.split('-',expand=True)
elim1.head()

,SHOW,SEASON,CONTESTANT,TYPE,VALUE,ELIM_DATE,NUMBER
0,Bachelorette,13,13_BRYAN_A,ELIMINATION-1,R1,ELIMINATION,1
1,Bachelorette,13,13_PETER_K,ELIMINATION-1,NaN,ELIMINATION,1
2,Bachelorette,13,13_ERIC_B,ELIMINATION-1,NaN,ELIMINATION,1
3,Bachelorette,13,13_DEAN_U,ELIMINATION-1,NaN,ELIMINATION,1
4,Bachelorette,13,13_ADAM_G,ELIMINATION-1,NaN,ELIMINATION,1


In [5]:
elim1.columns

Index(['SHOW', 'SEASON', 'CONTESTANT', 'TYPE', 'VALUE', 'ELIM_DATE', 'NUMBER'], dtype='object')

In [6]:
elim1.VALUE.unique()

array(['R1', nan, 'E', '1', 'EQ', 'EF', 'EU', 'R', '2', 'ED', '3', '4',
       '5', '6', 'W', '7', '8', '9', '10', 'D9', 'D6', 'D5', 'D1', 'D10',
       'D8', 'D7', 'D14', 'D13', 'D12', 'D11', 'D15', 'D6, D12',
       'D1, D12', 'D4', 'D2', 'D3'], dtype=object)

In [15]:
elim1[(elim1.VALUE == 'W') & (elim1.SHOW == 'Bachelorette')]

,SHOW,SEASON,CONTESTANT,TYPE,VALUE,ELIM_DATE,NUMBER
4925,Bachelorette,01,01_RYAN_S,ELIMINATION-6,W,ELIMINATION,6
5819,Bachelorette,02,02_IAN_M,ELIMINATION-7,W,ELIMINATION,7
6687,Bachelorette,04,04_JESSE_C,ELIMINATION-8,W,ELIMINATION,8
6713,Bachelorette,03,03_JERRY_F,ELIMINATION-8,W,ELIMINATION,8
7419,Bachelorette,11,11_SHAWN_B,ELIMINATION-9,W,ELIMINATION,9
8280,Bachelorette,13,13_BRYAN_A,ELIMINATION-10,W,ELIMINATION,10
8312,Bachelorette,12,12_JORDAN_R,ELIMINATION-10,W,ELIMINATION,10
8366,Bachelorette,10,10_JOSH_M,ELIMINATION-10,W,ELIMINATION,10
8392,Bachelorette,09,09_CHRIS_S,ELIMINATION-10,W,ELIMINATION,10
8418,Bachelorette,08,08_JEF_H,ELIMINATION-10,W,ELIMINATION,10


In [107]:
ette13 = elim1[(elim1.SEASON == '13') & (elim1.SHOW == 'Bachelorette')]
ette13.head(31)

,SHOW,SEASON,CONTESTANT,TYPE,VALUE,ELIM_DATE,NUMBER
0,Bachelorette,13,13_BRYAN_A,ELIMINATION-1,R1,ELIMINATION,1
1,Bachelorette,13,13_PETER_K,ELIMINATION-1,NaN,ELIMINATION,1
2,Bachelorette,13,13_ERIC_B,ELIMINATION-1,NaN,ELIMINATION,1
3,Bachelorette,13,13_DEAN_U,ELIMINATION-1,NaN,ELIMINATION,1
4,Bachelorette,13,13_ADAM_G,ELIMINATION-1,NaN,ELIMINATION,1
5,Bachelorette,13,13_MATT_M,ELIMINATION-1,NaN,ELIMINATION,1
6,Bachelorette,13,13_ALEX_B,ELIMINATION-1,NaN,ELIMINATION,1
7,Bachelorette,13,13_WILL_G,ELIMINATION-1,NaN,ELIMINATION,1
8,Bachelorette,13,13_KENNY_L,ELIMINATION-1,NaN,ELIMINATION,1
9,Bachelorette,13,13_ANTHONY_B,ELIMINATION-1,NaN,ELIMINATION,1


In [ ]:
# ette13.loc['NUMBER'] = ette13['NUMBER'].apply(pd.to_numeric, errors='coerce')
pd.to_numeric(ette13['NUMBER'])

In [45]:
elimrounds = ette13.NUMBER.max()
elimrounds

10.0

In [129]:
w1_cont = pd.unique(ette13.CONTESTANT) #select unique list of contestants 
w1_cont = [name for name in w1_cont if str(name) != 'nan'] #remove nan
len(w1_cont)

31

In [192]:
w1_cont_ID = {} #dictionary that contains numerical ID for contestants
ID = 0
for name in w1_cont:
    w1_cont_ID[name] = ID
    ID += 1
w1_cont_ID

{'13_BRYAN_A': 0,
 '13_PETER_K': 1,
 '13_ERIC_B': 2,
 '13_DEAN_U': 3,
 '13_ADAM_G': 4,
 '13_MATT_M': 5,
 '13_ALEX_B': 6,
 '13_WILL_G': 7,
 '13_KENNY_L': 8,
 '13_ANTHONY_B': 9,
 '13_JOSIAH_G': 10,
 '13_LEE_G': 11,
 '13_IGGY_R': 12,
 '13_JONATHAN_T': 13,
 '13_JACK_S': 14,
 '13_BRADY_E': 15,
 '13_BRYCE_P': 16,
 '13_DIGGY_M': 17,
 '13_FRED_J': 18,
 '13_BLAKE_E': 19,
 '13_JAMEY_K': 20,
 '13_LUCAS_Y': 21,
 '13_DEMARIO_J': 22,
 '13_BLAKE_K': 23,
 '13_GRANT_H': 24,
 '13_JEDIDIAH_B': 25,
 '13_KYLE_S': 26,
 '13_MICHAEL_B': 27,
 '13_MILTON_L': 28,
 '13_MOHIT_S': 29,
 '13_ROB_H': 30}

In [190]:
ette13_winner = ette13[ette13.VALUE == 'W'].CONTESTANT.values
ette13_winner_lst =[]
for winner in ette13_winner:
    break
ette13_winner_lst = [w1_cont_ID[winner]] * (int(elimrounds)+1)
print(ette13_winner)
print(ette13_winner_lst)


['13_BRYAN_A']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [79]:
#function returns name of random person from input contestant list
def randbaseline(cont_list):
    num_cont = len(cont_list)
    ran_cont_list = random.sample(cont_list, len(cont_list)) #shuffles input list
    win_num = random.randint(0,num_cont-1) #set seed??
    return ran_cont_list[win_num]
    

In [137]:
#function returns the updated contestant list after elimination occurs
def getcont(week,cont_list,data):
    numelim = 'ELIMINATION-' + week
    elim = data[(data.TYPE == numelim) & ((data.VALUE == 'E')|(data.VALUE == 'EU')|(data.VALUE == 'ED')|
       (data.VALUE == 'EF'))].CONTESTANT.values
    new_cont_list = cont_list.copy()
    new_cont_list = [elem for elem in new_cont_list if elem not in elim ]
    return new_cont_list
    
    

In [145]:
elimdict = {}
contlist = w1_cont
elimdict[1] = contlist
#loops through each elimination week and creates a dict of the updated contestant lists
for week in range(1,int(elimrounds)+1): 
    elimdict[week+1] = getcont(str(week),contlist,ette13)
    contlist = elimdict[week+1]
# elimdict

In [147]:
pred_winner_dict = {}
for key in elimdict.keys():
    pred_winner_dict[key] = randbaseline(elimdict[key])
pred_winner_dict

{1: '13_JACK_S',
 2: '13_BRYAN_A',
 3: '13_ANTHONY_B',
 4: '13_MATT_M',
 5: '13_ADAM_G',
 6: '13_ALEX_B',
 7: '13_MATT_M',
 8: '13_DEAN_U',
 9: '13_PETER_K',
 10: '13_PETER_K',
 11: '13_BRYAN_A'}

In [191]:
pred_winner = []
for key in pred_winner_dict.keys():
    pred_winner.append(w1_cont_ID[pred_winner_dict[key]])
print(pred_winner)
print(ette13_winner_lst)

[14, 0, 9, 5, 4, 6, 5, 3, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [187]:
f1_score(ette13_winner_lst, pred_winner, average='micro')

0.18181818181818182

In [193]:
ette12 = elim1[(elim1.SEASON == '12') & (elim1.SHOW == 'Bachelorette')]
pd.to_numeric(ette13['NUMBER'])
elimrounds12 = ette12.NUMBER.max()
w1_cont12 = pd.unique(ette12.CONTESTANT) #select unique list of contestants 
w1_cont12 = [name for name in w1_cont12 if str(name) != 'nan'] #remove nan
len(w1_cont12)

26

In [195]:
w1_cont_ID12 = {} #dictionary that contains numerical ID for contestants
ID = 0
for name in w1_cont12:
    w1_cont_ID12[name] = ID
    ID += 1

In [196]:
ette12_winner = ette12[ette12.VALUE == 'W'].CONTESTANT.values
ette12_winner_lst =[]
for winner in ette12_winner:
    break
ette12_winner_lst = [w1_cont_ID12[winner]] * (int(elimrounds12)+1)
print(ette12_winner)
print(ette12_winner_lst)

['12_JORDAN_R']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [197]:
elimdict12 = {}
contlist12 = w1_cont12
elimdict12[1] = contlist12
#loops through each elimination week and creates a dict of the updated contestant lists
for week in range(1,int(elimrounds12)+1): 
    elimdict12[week+1] = getcont(str(week),contlist12,ette12)
    contlist12 = elimdict12[week+1]
# elimdict

In [198]:
pred_winner_dict12 = {}
for key in elimdict12.keys():
    pred_winner_dict12[key] = randbaseline(elimdict12[key])
pred_winner_dict12

{1: '12_WELLS_A',
 2: '12_ALI_Z',
 3: '12_VINCENT_V',
 4: '12_WELLS_A',
 5: '12_DEREK_P',
 6: '12_WELLS_A',
 7: '12_ROBBY_H',
 8: '12_LUKE_P',
 9: '12_ROBBY_H',
 10: '12_ROBBY_H'}

In [200]:
pred_winner12 = []
for key in pred_winner_dict12.keys():
    pred_winner12.append(w1_cont_ID12[pred_winner_dict12[key]])
print(pred_winner12)
print(ette12_winner_lst)

[7, 16, 8, 7, 6, 7, 1, 3, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [201]:
f1_score(ette12_winner_lst, pred_winner12, average='micro')

0.0